# Load ascii file in memory

In [8]:
ascii_file_dir_path = '../data/input/bd_alti/'
ascii_file_name = 'BDALTIV2_25M_FXX_0825_6525_MNT_LAMB93_IGN69.asc'

with open(ascii_file_dir_path + ascii_file_name, 'r') as ascii_file:
    ncols = int(ascii_file.readline().replace('ncols', '').replace(' ', '').replace('\n', ''))
    nrows = int(ascii_file.readline().replace('nrows', '').replace(' ', '').replace('\n', ''))
    xllcorner = float(ascii_file.readline().replace('xllcorner', '').replace(' ', '').replace('\n', ''))
    yllcorner = float(ascii_file.readline().replace('yllcorner', '').replace(' ', '').replace('\n', ''))
    cellsize = int(float(ascii_file.readline().replace('cellsize', '').replace(' ', '').replace('\n', '')))
    NODATA_value = float(ascii_file.readline().replace('NODATA_value', '').replace(' ', '').replace('\n', ''))
    altitudes_table = []
    for line in ascii_file.readlines():
        raw_altitudes_line = line.split(' ')
        raw_altitudes_line.remove('')
        altitudes_table.append([float(alt) for alt in raw_altitudes_line])

# Deduce Bounding Box coordinates
Coordinates system : Lambert93 (xll, yll)

In [12]:
ascii_west_bound_xll = xllcorner
ascii_north_bound_yll = yllcorner
ascii_east_bound_xll = xllcorner + cellsize * ncols
ascii_south_bound_yll = yllcorner - cellsize * nrows

print('ascii_west_bound_xll : ' + str(ascii_west_bound_xll))
print('ascii_east_bound_xll : ' + str(ascii_east_bound_xll))
print('ascii_north_bound_yll : ' + str(ascii_north_bound_yll))
print('ascii_south_bound_yll : ' + str(ascii_south_bound_yll))


ascii_west_bound_xll : 824987.5
ascii_east_bound_xll : 849987.5
ascii_north_bound_yll : 6500012.5
ascii_south_bound_yll : 6475012.5


# Load shapefile Bounding Box coordinates from metadata
Coordinates system : WSG84 (longitude, latitude)

In [17]:
import xml.etree.ElementTree as ET
shapefile_dir_path = '../data/input/bd_carto/'
metadata_file_name = 'metadata.xml'
tree = ET.parse(shapefile_dir_path + metadata_file_name)
root = tree.getroot()

namespaces = {
                'gmd':'{http://www.isotc211.org/2005/gmd}',
                'gco':'{http://www.isotc211.org/2005/gco}'
            }

EX_GeographicBoundingBox_element = list(root.iter(namespaces['gmd']+'EX_GeographicBoundingBox'))[0]

west_bound_longitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'westBoundLongitude'))[0]
west_bound_longitude_decimal_element = list(west_bound_longitude_element.iter(namespaces['gco']+'Decimal'))[0]
shp_west_bound_longitude = float(list(west_bound_longitude_decimal_element.itertext())[0])
print('shp_west_bound_longitude: ' + str(shp_west_bound_longitude))

east_bound_longitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'eastBoundLongitude'))[0]
east_bound_longitude_decimal_element = list(east_bound_longitude_element.iter(namespaces['gco']+'Decimal'))[0]
shp_east_bound_longitude = float(list(east_bound_longitude_decimal_element.itertext())[0])
print('shp_east_bound_longitude: ' + str(shp_east_bound_longitude))

south_bound_latitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'southBoundLatitude'))[0]
south_bound_latitude_decimal_element = list(south_bound_latitude_element.iter(namespaces['gco']+'Decimal'))[0]
shp_south_bound_latitude = float(list(south_bound_latitude_decimal_element.itertext())[0])
print('shp_south_bound_latitude: ' + str(shp_south_bound_latitude))

north_bound_latitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'northBoundLatitude'))[0]
north_bound_latitude_decimal_element = list(north_bound_latitude_element.iter(namespaces['gco']+'Decimal'))[0]
shp_north_bound_latitude = float(list(north_bound_latitude_decimal_element.itertext())[0])
print('shp_north_bound_latitude: ' + str(shp_north_bound_latitude))

shp_west_bound_longitude: 2.0629
shp_east_bound_longitude: 7.1856
shp_south_bound_latitude: 44.1155
shp_north_bound_latitude: 46.8043


# Load shapefile in memory

# Convert WSG84 coordinates to Lamb93